# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf

from data_loader import MyDataLoader
from neural_nets import NeuralNets
from helper_funcs import *


%load_ext autoreload 
%autoreload 2

norm = (128,128)

dl = MyDataLoader("combine2", norm)
nn = NeuralNets(norm)

In [ ]:
dl.normalize_train_data(norm_size=norm)

try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()
dl.numpy_dump(X, "X")
dl.numpy_dump(y, "y")

In [2]:
# X = np.squeeze(dl.numpy_load("X"))
X = dl.numpy_load("X") # 0-1
y = dl.numpy_load("y") # -1 - 1
X_lab = X
y_lab = map_to(y)

In [4]:
print(np.max(y))
print(np.min(y))
print(np.max(y_lab))
print(np.min(y_lab))
test = map_from(y_lab, y)
print(np.max(test))
print(np.min(test))

0.758
-0.836
1.0
0.0
0.758
-0.836


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [ ]:
# from random import randint
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

# ?GANS svona
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

log_dir = "logs/fit/" + "model_e"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#model = load_model("model_e")
model = nn.autoencoder()
model.compile(optimizer=RMSprop(), loss=MeanSquaredError())


percent = 0.6
size = X_train.shape[0]
for i in range(10):
    rand_array = np.random.choice(size, int(percent*size))
    X_bla = X_train[rand_array]
    y_bla = y_train[rand_array]
    model.fit(X_bla, y_bla, batch_size = 64, epochs = 5, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])#, callbacks=[callback])
#save_model(model, "model_e", brave=True)



In [ ]:
%tensorboard --logdir logs/fit/

In [10]:
from keras.preprocessing.image import ImageDataGenerator



data_gen_args = dict(rotation_range=30, fill_mode='nearest',
                     width_shift_range=0.2, height_shift_range=0.2,
                     horizontal_flip=True, vertical_flip=True,
                     shear_range=0.2
)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1337
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)

image_generator = image_datagen.flow(X_train, seed = seed, batch_size=64)
mask_generator = image_datagen.flow(y_train, seed = seed, batch_size=64)

train_generator = zip(image_generator, mask_generator)

model = nn.model_e()
# model = load_model("model_e")
model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])

model.fit(train_generator, epochs=100, steps_per_epoch=128, validation_data=(X_val, y_val))


C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\keras_preprocessing\image\image_data_generator.py:938: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (8145, 128, 128, 2) (2 channels).
  warnings.warn(
C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\keras_preprocessing\image\numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (8145, 128, 128, 2) (2 channels).
  warnings.warn('NumpyArrayIterator is set to use the '


Epoch 1/100
100/100 [==============================] - 12s 116ms/step - loss: 0.0157 - accuracy: 0.8426 - val_loss: 0.0134 - val_accuracy: 0.8330
Epoch 2/100
 23/100 [=====>........................] - ETA: 8s - loss: 0.0139 - accuracy: 0.8291

KeyboardInterrupt: 

In [ ]:
save_model(model, "model_e_3", brave=True)


In [ ]:
model = load_model("model_e")
rand_array = np.random.choice(X_test.shape[0], 11)
pred = model.predict(X_test[rand_array])
show_images(X_test[rand_array], map_from(pred, y))

In [ ]:
print(pred.shape)
print(np.max(pred))
print(np.min(pred))
new = map_from(pred, y_train)
print(np.max(new))
print(np.min(new))
# show_images(X_test[rand_array], map_from(pred, y_train))

In [ ]:
print(X_lab.shape)
print(y_lab.shape)
print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))


In [ ]:
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True,
                             brightness_range=[0.4,1.5],
                             
)

In [ ]:
from helper_funcs import *
arr = y_train[:10]
print(type(arr[0,0,0,0]))
print(np.max(arr))
print(np.min(arr))

test = map_to(arr)
print(type(test[0,0,0,0]))
print(np.max(test))
print(np.min(test))

In [ ]:
baka = map_from(test, arr)
print(type(baka[0,0,0,0]))
print(np.max(baka))
print(np.min(baka))

In [ ]:

images = np.zeros(shape=(10, 128, 128, 3))
images[:,:,:,0] = np.squeeze(X_train[:10], -1)
images[:,:,:,1:] = y_train[:10]
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True
)
image_it = datagen.flow(images, batch_size=10, shuffle=True)

X_next = image_it.next()[:,:,:,0]
y_next = image_it.next()[:,:,:,1:]
X_bla = np.expand_dims(np.array(X_next), -1)
y_bla = np.array(y_next)
print(X_bla.shape)
print(y_bla.shape)
print(np.max(y_bla))
print(np.min(y_bla))
y_bla = map_from(y_bla, y)
print(np.max(y_bla))
print(np.min(y_bla))
show_images(X_bla, y_bla)